### Trade Demo

#### Goals:
- Login to the Canada domain.
- Select the dataset.
- Cacluate the sum of total of good imported to Egypt.
- Publish the result
- Download the results

### Step 1: Login into the Canada domain

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# As a Data Scientist we want to perform some analysis on the trade dataset available on the Canada domain.
# Our goal is to calculate the sum total of the goods imported (imports + re-imports) to Egypt

import syft as sy

# Let's login into the canada domain
canada_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)

Connecting to http://localhost:8081... done! 	 Logging into adp... done!


### Step 2: Select the dataset

In [3]:
# Awesome !!! We're logged into the domain node
# Let's quickly list all the datasets available on the Canada domain

canada_domain_node.datasets.pandas

<bound method RequestAPI.pandas of <syft.core.node.common.client_manager.dataset_api.DatasetRequestAPI object at 0x147780850>>

In [4]:
canada_domain_node.store.pandas

,ID,Tags,Description,object_type
0,<UID: e8b9b611a06b40b9bb2e590f4dac56f1>,[#feb2020],,<class 'syft.core.tensor.tensor.Tensor'>
1,<UID: 2185cb8c86a34a08987792104b1cb939>,[],,<class 'syft.lib.python.Int'>
2,<UID: 41dc5e2a20f3410784d4562ea01c050e>,[],,<class 'syft.lib.python.Int'>
3,<UID: 180039236ce24f4d9abb65a837bba092>,[],,<class 'syft.lib.python.Int'>
4,<UID: 3c9934f1154a499e921fd76262ebd792>,[],,<class 'syft.lib.python.Int'>
...,...,...,...,...
151,<UID: 284b3d680e9e45c1893550ad314c3bfb>,[],,<class 'syft.lib.python.Float'>
152,<UID: e449cfdb8aaf4ff7b264388c4bb996b0>,[],,<class 'syft.lib.python.Tuple'>
153,<UID: df0b99fb59c741f2bb7b35707b95915e>,[],,<class 'syft.lib.python.Tuple'>
154,<UID: d725354714b644deb9b1b61ac18315ee>,[],,<class 'syft.lib.python.Int'>


In [5]:
feb2020 = canada_domain_node.store[0]

# Alternatively, select the from the dataset
# feb2020 = canada_domain_node.datasets[0]
# feb2020.tags

In [6]:
# Let's try to get the whole dataset itself.
feb2020.request(reason="Access whole dataset")

#### STOP: Return to Data Owner-Canada.ipynb - STEP 8!!

### Step 3: Try to download the whole dataset

In [7]:
# Let's see if our request was accepted
feb2020.get()

[2021-08-12T17:01:38.942919-0400][CRITICAL][logger]][46552] You do not have permission to .get() Object with ID: <UID: e8b9b611a06b40b9bb2e590f4dac56f1>Please submit a request.


AuthorizationException: You do not have permission to .get() Object with ID: <UID: e8b9b611a06b40b9bb2e590f4dac56f1>Please submit a request.

In [ ]:
# Seems, like the request to download the whole dataset was denied by the Data Owner.

### Step 4: Private - Public Mutiplication

In [8]:
dataset_columns = [
    "Classification",
    "Year",
    "Period",
    "Period Desc.",
    "Aggregate Level",
    "Is Leaf Code",
    "Trade Flow Code",
    "Trade Flow",
    "Reporter Code",
    "Reporter",
    "Reporter ISO",
    "Partner Code",
    "Partner",
    "Partner ISO",
    "Commodity Code",
    "Commodity",
    "Qty Unit Code",
    "Qty Unit",
    "Qty",
    "Netweight (kg)",
    "Trade Value (US$)",
    "Flag",
]

# Since the columns are represented by indicies in the private tensor,
# let's create a column to index map.
column_to_index_map = {c: i for i, c in enumerate(dataset_columns)}
column_to_index_map

{'Classification': 0,
 'Year': 1,
 'Period': 2,
 'Period Desc.': 3,
 'Aggregate Level': 4,
 'Is Leaf Code': 5,
 'Trade Flow Code': 6,
 'Trade Flow': 7,
 'Reporter Code': 8,
 'Reporter': 9,
 'Reporter ISO': 10,
 'Partner Code': 11,
 'Partner': 12,
 'Partner ISO': 13,
 'Commodity Code': 14,
 'Commodity': 15,
 'Qty Unit Code': 16,
 'Qty Unit': 17,
 'Qty': 18,
 'Netweight (kg)': 19,
 'Trade Value (US$)': 20,
 'Flag': 21}

In [9]:
# Let's calculate the value of the goods imported from Egypt.
# Patner Code for Egypt is : 818
partner_code_for_egypt = 818
trade_flow_codes_map = {
    "imports": 1,
    "exports": 2,
    "re-exports":3,
    "re-imports":4,
}

In [10]:
# Breaking it down
partner_codes = feb2020[:, column_to_index_map["Partner Code"]]
trade_flow_code = feb2020[:, column_to_index_map["Trade Flow Code"]]

egypt_mask = partner_codes == partner_code_for_egypt
imports_mask = trade_flow_code == trade_flow_codes_map["imports"]



In [11]:
# Let's calculate the good imported from Egypt
value_of_goods_imported_from_egypt_mask = egypt_mask * imports_mask

In [12]:
# Let's continue with the analysis remotely.
# Let's calculate the value of the goods import from Egypt.

value_of_goods_reimported_from_egypt_mask = (
    feb2020[:, column_to_index_map["Partner Code"]] == partner_code_for_egypt
) * (feb2020[:, column_to_index_map["Trade Flow Code"]] == trade_flow_codes_map["re-imports"])

# Selecting Trade values for imports and re-imports (Private - Private Multiplication)
value_of_goods_imported_from_egypt = feb2020[:, column_to_index_map["Trade Value (US$)"]] * value_of_goods_imported_from_egypt_mask
value_of_goods_reimported_from_egypt = feb2020[:, column_to_index_map["Trade Value (US$)"]] * value_of_goods_reimported_from_egypt_mask

# Let's scale the trade value by 1000 - (Private - Public Multiplication)
scaled_value_of_goods_imported_from_egypt = value_of_goods_imported_from_egypt * (0.001)
scaled_value_of_goods_reimported_from_egypt = value_of_goods_reimported_from_egypt * (0.001)

### Step 5: Private - Private Addition

In [14]:
# sigma -> Amount of noise added to the response.
# Larger the sigma, smaller the budget spent.

# Calculating th total imported goods (Private - Private Addition)
private_total_imported_goods_value = scaled_value_of_goods_imported_from_egypt + scaled_value_of_goods_reimported_from_egypt
private_total_imported_goods_value = private_total_imported_goods_value.sum()

In [17]:
public_total_imported_goods_value = private_total_imported_goods_value.publish(client=canada_domain_node ,sigma=1.0)

a
b
c
d


In [ ]:
print(private_total_imported_goods_value.id_at_location)
print(public_total_imported_goods_value.id_at_location)

In [ ]:
private_total_imported_goods_value.update_searchability(pointable=True)

### Step 6: Download Results

In [ ]:
# Let's download the results
canada_domain_node.store

In [16]:
total_imported_goods_value = public_total_imported_goods_value.get()
print(f"Total trade value($) of the goods Imported from Egypt to Canada: ${total_imported_goods_value}")

[2021-08-12T17:02:29.530367-0400][CRITICAL][logger]][46552] UnknownPrivateException has been triggered.


UnknownPrivateException: UnknownPrivateException has been triggered.

In [ ]:
# b65f0512fd814ec58520b9cf009cada1>

*Great !!! We successfully, finished our analysis.*

# CONGRATULATIONS!!!

You've finished the demo!